In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import warnings
from keras.layers import Input, Embedding, Flatten, Dot, Dense , Concatenate , StringLookup, BatchNormalization,Dropout 
from keras.models import Model , load_model
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
tf.random.set_seed(42)
warnings.filterwarnings('ignore')
import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import warmstart_embedding_matrix

dataPath = 'C:\\Users\\Sigma\\ecommerce\\allData.csv'
ratingPath = 'C:\\Users\\Sigma\\ecommerce\\both3.csv'
new = pd.read_csv(r'C:\Users\Sigma\ecommerce\new.csv')
model2 = 1

ratings_dataset = pd.read_csv(ratingPath)
n_users = len(ratings_dataset['userID'].unique())
n_items = len(ratings_dataset['itemID'].unique())

In [7]:

    userId_lookup = StringLookup(vocabulary = ratings_dataset['userID'].unique(), mask_token = None )
    asin_lookup = StringLookup(vocabulary = ratings_dataset['itemID'].unique(), mask_token = None )


    item_input = Input(shape=[1],name='item-input'  , dtype= object)
    user_input = Input(shape=[1], name='user-input' , dtype= object)

    # MLP Embeddings
    item_input1 = asin_lookup(item_input)
    user_input1 = userId_lookup(user_input)

    item_embedding_mlp = Embedding(n_items + 1, 16, name='item-embedding-mlp'  )(item_input1)
    item_vec_mlp = Flatten(name='flatten-item-mlp')(item_embedding_mlp)

    user_embedding_mlp = Embedding(n_users + 1, 16, name='user-embedding-mlp' )(user_input1)
    user_vec_mlp = Flatten(name='flatten-user-mlp')(user_embedding_mlp)

    # MF Embeddings
    item_embedding_mf = Embedding(n_items + 1, 16, name='item-embedding-mf' , embeddings_regularizer='L2' )(item_input1)
    item_vec_mf = Flatten(name='flatten-item-mf')(item_embedding_mf)

    user_embedding_mf = Embedding(n_users + 1, 16, name='user-embedding-mf' , embeddings_regularizer='L2' )(user_input1)
    user_vec_mf = Flatten(name='flatten-user-mf')(user_embedding_mf)

    # MLP layers

    concat = Concatenate()([item_vec_mlp, user_vec_mlp])
    concat_dropout = Dropout(0.5)(concat)

    fc_1 = Dense(128, name='fc-1', activation='relu')(concat_dropout)
    fc_1_bn = BatchNormalization(name='batch-norm-1')(fc_1)
    fc_1_dropout = Dropout(0.5)(fc_1_bn)

    fc_2 = Dense(64, name='fc-2', activation='relu')(fc_1_dropout)
    fc_2_bn = BatchNormalization(name='batch-norm-2')(fc_2)
    fc_2_dropout = Dropout(0.5)(fc_2_bn)


    # fc_3 = Dense(32, name='fc-3', activation='relu')(fc_2_dropout)
    # fc_3_bn = BatchNormalization(name='batch-norm-3')(fc_3)
    # fc_3_dropout = Dropout(0.5)(fc_3_bn)


    # Prediction from both layers
    pred_mlp = Dense(16, name='pred-mlp', activation='relu' )(fc_2_dropout)



    pred_mf = Dot(name="Dot-Product", axes=1)([item_vec_mf, user_vec_mf])
    combine_mlp_mf = Concatenate()([pred_mf, pred_mlp])

    # Final prediction
    result = Dense(1, name='result', activation='relu' )(combine_mlp_mf)

    model2 = Model([user_input, item_input], result)
    model2.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss= tf.keras.losses.mean_absolute_error  , metrics= tf.keras.metrics.RootMeanSquaredError(
        name="root_mean_squared_error", dtype=None))
    import os
    if os.path.exists(r'C:\Users\Sigma\ecommerce\embedding_model.h5'):
        model2 = load_model(r'C:\Users\Sigma\ecommerce\embedding_model.h5')
    else:
        #history = model2.fit([train.userId, train.asin], labels, batch_size=64  , epochs=10 ,  verbose=1)
        history = model2.fit([ratings_dataset.userID, ratings_dataset.itemID], ratings_dataset['rating'] , batch_size=64  , epochs= 10 ,  verbose=1 )
        model2.save(r'C:\Users\Sigma\ecommerce\embedding_model.h5')
    print(len(model2.layers[15].get_weights()[0]))

12712


In [8]:
def newEmbedModel() :
    
    import keras
    import tensorflow as tf
    import numpy as np
    from tensorflow.keras.utils import warmstart_embedding_matrix




    base_vocabulary_user = ratings_dataset['userID'].unique()
    base_vocabulary_item = ratings_dataset['itemID'].unique()
    # old_text_vectorization_layer.get_vocabulary()
    new = pd.read_csv(r'C:\Users\Sigma\ecommerce\new.csv')
    new_vocabulary_user = new['userID'].unique()
    new_vocabulary_item = new['itemID'].unique()
    # new_text_vectorization_layer.get_vocabulary()
    # get previous embedding layer weights


    embedding_weights_base_item1 = model2.get_layer('item-embedding-mlp').get_weights()[0]
    embedding_weights_base_item2 = model2.get_layer('item-embedding-mf').get_weights()[0]
    embedding_weights_base_user1 = model2.get_layer('user-embedding-mlp').get_weights()[0]
    embedding_weights_base_user2 = model2.get_layer('user-embedding-mf').get_weights()[0]



    warmstarted_embedding1 = warmstart_embedding_matrix(
                                  base_vocabulary_user,
                                  new_vocabulary_user,
                                  base_embeddings=embedding_weights_base_user1,
                                  new_embeddings_initializer="uniform")
    updated_embedding_variable1 = tf.Variable(warmstarted_embedding1)

    # update embedding layer weights ####################################33
    model2.layers[5].embeddings = updated_embedding_variable1


    warmstarted_embedding2 = warmstart_embedding_matrix(
                                  base_vocabulary_user,
                                  new_vocabulary_user,
                                  base_embeddings=embedding_weights_base_user2,
                                  new_embeddings_initializer="uniform")
    updated_embedding_variable2 = tf.Variable(warmstarted_embedding2)

    # update embedding layer weights  ####################################
    model2.layers[15].embeddings = updated_embedding_variable2


    warmstarted_embedding3 = warmstart_embedding_matrix(
                                  base_vocabulary_item,
                                  new_vocabulary_item,
                                  base_embeddings=embedding_weights_base_item1,
                                  new_embeddings_initializer="uniform")
    updated_embedding_variable3 = tf.Variable(warmstarted_embedding3)

    # update embedding layer weights  ####################################
    model2.layers[4].embeddings = updated_embedding_variable3


    warmstarted_embedding4 = warmstart_embedding_matrix(
                                  base_vocabulary_item,
                                  new_vocabulary_item,
                                  base_embeddings=embedding_weights_base_item2,
                                  new_embeddings_initializer="uniform")
    updated_embedding_variable4 = tf.Variable(warmstarted_embedding4)

    # update embedding layer weights
    model2.layers[14].embeddings = updated_embedding_variable4

    new5 = pd.read_csv(r'C:\Users\Sigma\ecommerce\new1.csv')
    history = model2.fit([new5.userID, new5.itemID], new5['rating'] , batch_size=64  , epochs=35,  verbose=1 )
    new5 = new5[0:0]
    new5.to_csv(r'C:\Users\Sigma\ecommerce\new1.csv' , index=False)
    new.to_csv(r'C:\Users\Sigma\ecommerce\both3.csv' , index=False)
    print(len(model2.layers[15].get_weights()[0]))


In [9]:
model2 = load_model(r'C:\Users\Sigma\ecommerce\embedding_model.h5')
len(model2.get_layer('user-embedding-mlp').get_weights()[0])


12712

In [10]:
newEmbedModel()


Epoch 1/35


KeyError: in user code:

    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\engine\training.py", line 1027, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\optimizers\optimizer_experimental\optimizer.py", line 527, in minimize
        self.apply_gradients(grads_and_vars)
    File "C:\Users\Sigma\anaconda3\lib\site-packages\keras\optimizers\optimizer_experimental\optimizer.py", line 1140, in apply_gra